<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_groupchat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Generated Agent Chat: Group Chat 自動產生的代理聊天：群聊

AutoGen 提供由 LLM、工具或人員支援的可對話代理，可用於透過自動聊天集體執行任務。 該框架允許透過多代理對話使用工具和人類參與。
請在[此處](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat)尋找有關此功能的文件。

本筆記本根據https://github.com/microsoft/FLAML/blob/4ea686af5c3e8ff24d9076a7a626c8b28ab5b1d7/notebook/autogen_multiagent_roleplay_chat.ipynb修改

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install:
```bash
pip install pyautogen
```

In [105]:
%%capture --no-stderr
# %pip install "pyautogen>=0.2.3"

## Set your API Endpoint 設定您的 API 端點

[`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) 函數從環境變數或 json 檔案載入設定清單。

將調用的大模型改成 gpt-3.5，以節省費用。

In [1]:
import autogen

""" config_list_gpt = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
) """
config_list_gpt = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        },
    },
)

它首先查找環境變數“OAI_CONFIG_LIST”，該變數需要是有效的 json 字串。 如果未找到該變量，它將尋找名為“OAI_CONFIG_LIST”的 json 檔案。 它按型號過濾配置（您也可以按其他鍵過濾）。 根據過濾條件，僅 gpt-3.5 型號保留在清單中。

配置列表如下所示：
```python
config_list = [
    {
        "model": "gpt-3.5-turbo",
        "api_key": "sk-xxx"
    },
    {
        "model": "gpt-3.5-turbo-1106",
        "api_key": "sk-xxx"
    },
    {
        "model": "gpt-3.5-turbo-0613",
        "api_key": "sk-xxx"
    },
    {
        "model": "gpt-3.5-turbo-16k",
        "api_key": "sk-xxx"
    },
    {
        "model": "gpt-3.5-turbo-16k-0613",
        "api_key": "sk-xxx"
    }
]
```

您可以按照您喜歡的任何方式設定 config_list 的值。 請參閱此[筆記本](https://github.com/microsoft/autogen/blob/main/notebook/oai_openai_utils.ipynb)以取得不同方法的完整程式碼範例。

## Construct Agents 建立代理(智能體)
config_list_gpt 裡面過濾後還有 5 的 gpt-3.5。不確定下面指令執行後，在 autogen.AssistantAgent 設定 llm_config=llm_config 是會去掉用哪一個 gpt-3.5 ?

llm_config = {"config_list": config_list_gpt, "cache_seed": 42}


In [2]:
llm_config = {"config_list": config_list_gpt, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={"last_n_messages": 2, "work_dir": "groupchat"},
    human_input_mode="TERMINATE",
)
coder = autogen.AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)
pm = autogen.AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config=llm_config,
)
groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

## Start Chat 開始對話

並要求以繁體中文回應(And reponse in traditional chinese)

In [ ]:
user_proxy.initiate_chat(
    manager,
    message="Find a latest paper about gpt-4 on arxiv and find its potential applications in software. And reponse in traditional chinese.",
)
# type exit to terminate the chat